In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

In [27]:
# Cargar datos
data = pd.read_csv("../Data/MXvideos.csv", encoding='latin1')
titles = data['title'].astype(str)

# Vectorización TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(titles)

In [28]:
# Función para encontrar videos similares
def get_similar_videos(query, top_n=10):
    query_vec = vectorizer.transform([query])
    similarities = cosine_similarity(query_vec, X).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1]
    return data.iloc[top_indices][['title', 'channel_title', 'views']]
# Consulta ejemplo
query_video = "Best music hits 2024"
recommended_videos = get_similar_videos(query_video)

In [29]:
# Nube de palabras
wordcloud_text = ' '.join(titles)
wordcloud = WordCloud(width=1000, height=500, background_color='black', colormap='cool', max_words=100).generate(wordcloud_text)
wordcloud.to_file("wordcloud.png")  # Guardar imagen

In [30]:
# Histograma de longitud de títulos
plt.figure(figsize=(8, 4))
sns.histplot(data['title'].apply(lambda x: len(x.split())), bins=30, kde=True, color='purple')
plt.title("Distribución de la Longitud de los Títulos")
plt.xlabel("Número de Palabras")
plt.ylabel("Frecuencia")
plt.savefig("title_length.png")  # Guardar imagen
plt.close()

In [31]:
# Top 10 canales con más videos
top_channels = data['channel_title'].value_counts().head(10)
plt.figure(figsize=(8, 4))
sns.barplot(x=top_channels.values, y=top_channels.index, palette='magma', hue=top_channels.index, dodge=False)
plt.title("Top 10 Canales con Más Videos")
plt.xlabel("Número de Videos")
plt.savefig("top_channels.png")  # Guardar imagen
plt.close()

In [32]:
# Top 10 videos con más dislikes
if 'dislikes' in data.columns:
    top_disliked_videos = data[['title', 'dislikes']].sort_values(by='dislikes', ascending=False).head(10)
    plt.figure(figsize=(8, 4))
    sns.barplot(x=top_disliked_videos['dislikes'], y=top_disliked_videos['title'], palette='Reds_r', hue=top_disliked_videos['title'], legend=False)
    plt.title("Top 10 Videos con Más Dislikes")
    plt.xlabel("Número de Dislikes")
    plt.savefig("top_dislikes.png")  # Guardar imagen
    plt.close()

In [33]:
# 📌 CREAR REPORTE PDF (Versión mejorada)
def generate_pdf():
    pdf_filename = "YouTube_Analysis_Report.pdf"
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    c.setFont("Helvetica-Bold", 14)

    # Título
    c.drawString(200, 750, "Reporte de Análisis de Videos de YouTube")

    # Descripción
    c.setFont("Helvetica", 10)
    c.drawString(50, 730, f"Consulta: {query_video}")
    c.drawString(50, 715, "Este análisis muestra los videos más similares y estadísticas relevantes.")

    # Lista de videos recomendados
    y_position = 690
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y_position, "Top 10 Videos Recomendados:")

    c.setFont("Helvetica", 10)
    y_position -= 20
    for i, row in recommended_videos.iterrows():
        c.drawString(50, y_position, f"{i+1}. {row['title']} - {row['channel_title']} ({row['views']} views)")
        y_position -= 15
        if y_position < 100:  # Si se acaba el espacio, pasar a nueva página
            c.showPage()
            y_position = 750

    # Función para agregar imágenes y verificar espacio
    def add_image(image_path, title, y):
        c.showPage()  # Nueva página para cada imagen
        c.setFont("Helvetica-Bold", 12)
        c.drawString(50, 750, title)
        img = ImageReader(image_path)
        c.drawImage(img, 50, 450, width=500, height=250)

    # Insertar imágenes en nuevas páginas
    add_image("wordcloud.png", "Nube de Palabras", 450)
    add_image("title_length.png", "Distribución de Longitud de Títulos", 450)
    add_image("top_channels.png", "Top 10 Canales con Más Videos", 450)

    if 'dislikes' in data.columns:
        add_image("top_dislikes.png", "Top 10 Videos con Más Dislikes", 450)

    # Guardar PDF
    c.save()
    print(f"Reporte generado: {pdf_filename}")

# Ejecutar la generación del PDF
generate_pdf()

Reporte generado: YouTube_Analysis_Report.pdf
